# Tutoriel #5 - Geographic Web scraping

Dans cet partie, vous allez produire une carte interactive en HTML à partir de données extraites d'une page web. Pour ce faire, vous allez :
1. Extraire les données pertinentes d'un tableau présenté sur une page web.
2. Les stocker dans un format de données approprié.
3. Déclarer et paramétrer une carte.
4. Créer une couche de données pour y ajouter les points dont vous avez obtenu les coordonnées.

> <small>**Les données utilisées sont issues de la page [*Family: Tai-Kadai*](https://glottolog.org/resource/languoid/id/taik1256) de  [Glottolog](https://glottolog.org/)**\
Glottolog 4.5 edited by Hammarström, Harald & Forkel, Robert & Haspelmath, Martin & Bank, Sebastian
is licensed under a [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/).</small>

## Analyse visuelle de la page web

**Vous trouverez [sur cette page web](https://alxdrdelaporte.github.io/works/taik1256_glottolog_table.html) un tableau de données correspondant aux langues de la famille Tai-Kadai [(Wikipedia)](https://fr.wikipedia.org/wiki/Langues_ta%C3%AF-kada%C3%AF). Chaque ligne du tableau représente une langue. Pour chaque langue, les coordonnées géographiques de de la région d'où est originaire cette langue sont indiquées.**

La page web à scaper est représentée ci-dessous :

<iframe width="100%" height="700px" src="https://alxdrdelaporte.github.io/works/taik1256_glottolog_table.html"></iframe>

Ouvrez la page web avec votre navigateur. Puis, affichez le code source de cette page (`Ctrl U` sous Chrome). Analysez la structure de la page. Identifiez les différentes balises utilisées, ainsi que les identifiants.

Vous allez devoir parser<sup>*</sup> cette page automatiquement à l'aide d'un code Python.

> \* le **parsing** consiste à parcourir le contenu d'un texte ou d'un fichier en l'analysant pour vérifier sa syntaxe ou en extraire des éléments.

## Imports des modules utiles

Vous n'avez pas besoin de modifier la cellule ci-dessous, elle contient les modules et librairies Python dont vous aurez besoin pour cette partie du travail. Pour que le programme fonctionne, n'oubliez pas de l'exécuter quand même !

In [119]:
# Parsing HTML
from bs4 import BeautifulSoup as soup
# Client web
from urllib.request import urlopen as uReq
import urllib.request

La classe `BeautifuleSoup` du module `bs4` va permettre de réaliser l'analyse syntaxique de lapage web afin d'en extraire les éléments qui serviront à créer la cartographie des langues de la famille Tai-Kadai.

### Fonctions utiles

Vous pouvez également laisser telle quelle la cellule suivante.

Elle comporte 2 fonctions fournis pour faciliter votre travail :
1. `url_ok()` prend en paramètre une URL et vérifie si elle est accessible. Vous n'aurez pas besoin d'appeler directement cette fonction, elle sera utilisée par la fonction `good_soup()`.
2. `good_soup()` prend elle aussi une URL en paramètre. Si l'URL est accessible (= validée par `url_ok()`), cette fonction renvoie le code HTML parsé que vous pourrez ensuite parcourir.

In [120]:
def url_ok(url):
    """Teste si une URL est accessible, paramètre = URl (str)"""
    request = urllib.request.Request(url)
    request.get_method = lambda: 'HEAD'
    try:
        urllib.request.urlopen(request)
        return True
    except urllib.request.HTTPError:
        return False
    
    
def good_soup(url):
    """Parser le HTML d'une page web avec bs4, paramètre = URL (str)"""
    if url_ok(url):
        uClient = uReq(url)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, "html.parser")
        return page_soup

**Et maintenant, à vous de jouer !**

## Extraction du tableau de la page HTML

Afin d'explorer le code source d'une page web, il faut préalablement le parser pour pouvoir l'explorer automatiquement. C'est exactement ce que fait la fonction `good_soup()` présentée ci-dessus.

Appelez cette fonction sur l'URL de la page où se trouve le tableau, en associant le résultat à la variable `ma_soupe`.

In [121]:
# Appeler la fonction good_soup() sur l'URL
ma_soupe = good_soup("https://alxdrdelaporte.github.io/works/taik1256_glottolog_table.html")

S'il n'y a pas d'erreur, appeler `ma_soupe`ou utiliser `print(ma_soupe)` devrait afficher le code source de la page HTML.

In [ ]:
ma_soupe

La page ne contient pas que le tableau, il va falloir l'isoler du reste du contenu. 

Pour récupérer un élément unique sur une page, la librairie `bs4` ([documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc)) dispose d'une fonction `find()`. Elle peut par exemple prendre en argument :
* L'identifiant de l'élément avec la syntaxe `objetbs4.find(id="identifiant_element")`
* Le tag de l'élément avec la syntaxe `objetbs4.find("tag")`

Dans le second cas, si plusieurs éléments de la page comportent le même tag, seul le premier sera récupéré.

Ici, il n'y a qu'un tableau sur la page et celui-ci porte un identifiant, les deux solutions mèneront au même résultat. Le contenu de votre variable `ma_soupe` étant un objet `bs4`, vous pouvez appeler la fonction `find()` dessus.

Dans la cellule ci-dessous, appelez `find()` sur `ma_soupe` en renseignant l'argument de façon à obtenir le code HTML correspondant uniquement au tableau. Le résultat sera associé à la variable `tableau`.

In [123]:
# isoler le tableau, dont l'identifiant est tab_taik1256, dans le code source de la page (ma_soupe)
tableau = ma_soupe.find(id="tab_taik1256")

# autre solution possible avec le tag 
# tableau = ma_soupe.find("table")

Comme pour le parsing de la page dans son ensemble, vous pouvez vérifier qu'il n'y a pas de problème :

In [ ]:
# vérifier le contenu de la variable tableau
tableau

### Filtrer le contenu du tableau

Pour produire une carte, les seules données indispensables sont les coordonnées géographiques de chaque point. Il est nécessaire de les récupérer.

Par ailleurs, pour que les données restent interprétables, il serait aussi préférable de conserver une indication de ce à quoi correspond chaque paire de coordonnées, comme le nom et/ou l'identifiant de la langue.

Pour parcourir les lignes du tableau, il n'est pas possible d'utiliser `find("tr")` qui ne renverrait que la première ligne. Heureusement, la fonction `find_all()` s'utilise de façon similaire et renvoie tous les éléments correspondant à la requête. Le résultat de `find_all()` peut être parcouru avec une boucle `for`.

Complétez la cellule ci-dessous pour afficher chacune des lignes du tableau :

In [ ]:
for ligne in tableau.find_all("tr"):
    print(ligne)

In [ ]:
# corrigé : exclure la ligne d'étiquettes
for ligne in tableau.find("tbody").find_all("tr"):
    print(ligne)

La ligne d'étiquettes n'est pas une ligne de données, nous ne voulons pas la récupérer ! Si ce n'est pas fait, adaptez le code de la cellule précédente pour n'afficher que les lignes de données.

Vous l'aviez peut-être deviné en appliquant la consigne précédente, mais `find_all()` permet d'appliquer un index à ses résultats. En reprenant la boucle `for` qui parcourt les lignes du tableau, vous pouvez maintenant afficher uniquement les colonnes pertinentes du tableau.

**Note :** l'index débute à 0 et non à 1.

In [ ]:
# n'afficher que les lignes correspondant à la longitude, la latitude, le nom et l'identifiant
for ligne in tableau.find("tbody").find_all("tr"):
    longitude = ligne.find_all("td")[3]
    print(longitude)
    latitude = ligne.find_all("td")[4]
    print(latitude)
    nom = ligne.find_all("td")[2]
    print(nom)
    identifiant = ligne.find_all("td")[5]
    print(identifiant)

Très bien, les données que nous cherchons à récupérer sont bien là, mais elles sont encore entourées de balises `<td>`. Pour les supprimer, vous pouvez faire appel à l'attribut `.text` des objets `bs4`, qui correspond au contenu textuel de l'élément concerné. Par exemple :

In [ ]:
ma_soupe.find("h1").text

### Stocker les données dans un dictionnaire

Modifier la boucle `for` en appelant l'attribut `.text` des éléments permettrait d'afficher le texte dans la console, mais en l'état les données ne seront enregistrées nulle part et il ne sera donc pas possible de les réutiliser sans devoir les produire à nouveau.

La solution qui sera utilisée ici est la suivante :
* L'ensemble des lignes sera stockée dans une [liste](https://docs.python.org/fr/3.10/tutorial/datastructures.html#more-on-lists).
* Dans cette liste, chacune des lignes correspondra à un [dictionnaire](https://docs.python.org/fr/3.10/tutorial/datastructures.html#dictionaries), c'est-à-dire une liste associative.

Une liste vide est initiée avec `[]`.

Un dictionnaire vide s'initie avec `{}`. Pour déclarer un nouveau dictionnaire en ajoutant directement les données, la syntaxe est la suivante :

```
nom_dictionnaire = {"clé1": valeur1, "clé2": valeur2, ..., "cléX": valeurX}
```

Pour ajouter des données à un dictionnaire déjà créé, vous pouvez utiliser :


```
nom_dictionnaire["clé"]= valeur
```

Voici déjà la liste qui va accueillir l'ensemble des données.

In [129]:
# Liste qui va accueillir les dictionnaires
donnees = []

Il faut maintenant peupler cette liste, en lui ajoutant un dictionnaire pour chaque ligne extraite du tableau.

En vous aidant de la boucle `for` déjà écrite et de la [documentation](https://docs.python.org/fr/3.10/tutorial/datastructures.html), complétez la fonction ci-dessous pour obtenir la liste des données :

In [130]:
for ligne in tableau.find("tbody").find_all("tr"):
    dico_ligne = {
        "longitude": ligne.find_all("td")[3].text,
        "latitude": ligne.find_all("td")[4].text,
        "nom": ligne.find_all("td")[2].text,
        "id": ligne.find_all("td")[5].text
    }
    # ajouter dico_ligne à la liste donnees
    donnees.append(dico_ligne)

S'il n'y a pas d'erreur, le début du contenu de `donnees` devrait ressembler à ceci :

```
[{'longitude': '104.812',
  'latitude': '23.2384',
  'nom': 'Pubiao-Qabiao',
  'id': 'qabi1235'},
 {'longitude': '105.754',
  'latitude': '23.342',
  'nom': 'Yerong-Southern Buyang',
  'id': 'yero1238'},
```

Appelez la liste `donnees` pour vérifier qu'il n'y a pas de problème.

In [ ]:
# vérifier le contenu de donnees
donnees

## Construction de la représentation cartographique

Les informations utiles ayant été extraites de la page web, et placées dans un dictionnaire Python, vous allez procéder à leur représentation sur une carte.

### Import de la librairie `folium`

Pour produire une carte au format HTML, vous aurez besoin de la librairie `folium`. Celle-ci est importée dans la cellule ci-dessous, qui n'a pas besoin d'être modifiée.

In [132]:
import folium

### Mettre en place le fond de carte

Avant de représenter les données, il faut d'abord déclarer et paramétrer la carte, via une instance d'objet `Map` de la librairie `folium`. Vous pouvez éventuellement initialiser l'objet sans lui attribuer de paramètre, en l'associant à la variable `ma_carte`.

In [133]:
# déclarez la variable ma_carte en lui associant une instance d'objet folium Map() sans préciser de paramètre
ma_carte = folium.Map()

Pour obtenir un aperçu de la carte, il suffit d'appeler la variable dans laquelle elle est stockée.

In [ ]:
# prévisualiser la carte
ma_carte

Vous obtenez bien une carte, mais cette vue par défaut n'est ni satisfaisante ni optimale pour visualiser des données.

Déclarez à nouveau votre carte, en renseignant au moins les paramètres `location` (coordonnées par défaut), `zoom_start` (zoom par défaut, basé sur la valeur de `location`), et `tiles` (fond de carte par défaut).

Pour ce faire, vous pouvez vous aider de [cette page de la documentation de `folium`](https://python-visualization.github.io/folium/modules.html).

Afin de centrer la carte sur la bonne position géographique, il est nécessaire de **calculer le barycentre** (c'est-à-dire, la moyenne) de toutes les localisations présentes dans le dictionnaire.

In [ ]:
# Convertir les valeurs de longitude et latitude en float et calculer les moyennes
total_longitude = 0
total_latitude = 0
count = len(donnees)

for entry in donnees:
    total_longitude += float(entry['longitude'])
    total_latitude += float(entry['latitude'])

barycentre_longitude = total_longitude / count
barycentre_latitude = total_latitude / count

# Affichage des coordonnées du barycentre
print(f"Barycentre Longitude: {barycentre_longitude}")
print(f"Barycentre Latitude: {barycentre_latitude}")

In [136]:
# déclarer à nouveau la carte avec ses paramètres
ma_carte = folium.Map(
    location=[barycentre_latitude, barycentre_longitude],
    zoom_start=5,
    tiles='openstreetmap')

In [ ]:
# prévisualiser à nouveau la carte
ma_carte

IL est également intéressant d'ajouter un titre à la carte. 

In [ ]:
# ajouter un titre avec un IFrame (du HTML intégré)
title_html = '''
             <div style="position: fixed; 
                         top: 10px; left: 50%; 
                         transform: translateX(-50%);
                         font-size: 24px; font-family: Arial, sans-serif; 
                         background-color: rgba(255, 255, 255, 0.7); 
                         padding: 10px; border-radius: 5px;
                         color: black; z-index: 9999;">
             Cartographie des langues tai-kadai
             </div>
             '''
ma_carte.get_root().html.add_child(folium.Element(title_html))

# affichage de la carte
ma_carte

C'est (normalement) beaucoup mieux comme ça !

### Déclarer une couche de données
Il est temps d'ajouter à la carte les données extraites du tableau HTML. Voici comment procéder :
1. Déclarer une couche de données
2. Ajouter les données à la couche
3. Ajouter la couche à la carte

La couche de données correspond à un objet de `folium` nommé `FeatureGroup`. Vous trouverez ses différents paramètres dans [la documentation](https://python-visualization.github.io/folium/modules.html#folium.map.FeatureGroup), mais vous pouvez ici l'initialiser avec seulement le paramètre `name`. 

Pour pouvoir l'ajouter à la carte ensuite il est indispensable de stocker les informations de la couche de données dans une variable. Je vous conseille de la nommer de façon à identifier quelles sont les données correspondantes (vous pouvez par exemple reprendre l'identifiant Glottolog de la famille Tai-Kadai).

In [139]:
# déclarer une couche de données en renseignant le paramètre name, l'associer à une variable nommée explicitement
taik1256 = folium.FeatureGroup("taik1256")

### Reporter les coordonnées géographiques dans la couche de données

La couche de données existe maintenant dans le sens où une instance d'objet `FeatureGroup` a été créée, mais elle ne comporte aucune information (à part son nom), et n'est pas reliée à la carte.

Pour rappel, les coordonnées sont actuellement stockées dans la variable `donnees`, sous forme d'une liste de dictionnaires.

Celles-ci correspondent à une série de points. Dans le code, chaque point correspondra à un objet `folium.CircleMarker()` qui prendra en paramètre les coordonnées géographiques dans une liste `[latitude, longitude]`. 

Vous disposez en fait déjà de tout ce qu'il vous faut pour ajouter vos points à la couche de données.\
Sachant qu'un point est ajouté avec la fonction `.add_to(nom_de_la_couche_de_donnees)`, décommentez et complétez le code de la cellule suivante :

In [140]:
for ligne in donnees:
    langue_et_lien = f'<p><b>{ligne["nom"]}</b></p><a href="{URL}" target="_blank">Fiche Glottolog</a>'
    folium.CircleMarker(
        location=[ligne["latitude"], ligne["longitude"]],
        radius=7,  # Ajuster le rayon du cercle en pixels
        weight=2,  # Épaisseur de la bordure du cercle en pixels
        color='blue',  # Couleur du cercle
        fill=True,  # Remplir le cercle
        fill_color='blue',  # Couleur de remplissage
        fill_opacity=0.25,  # Opacité du remplissage
    ).add_to(taik1256)

### Ajouter la couche de données à la carte

Le plus dur est fait. La fonction `.add_to()` permet également d'ajouter une couche de données à une carte, en utilisant la syntaxe `nom_de_la_couche_de_donnees.add_to(nom_de_la_carte)`.

In [ ]:
# ajouter la couche de données à la carte
taik1256.add_to(ma_carte)

# visualiser la carte
ma_carte

Il ne reste plus qu'à sauvegarder cette carte au format HTML.

In [142]:
ma_carte.save("carte_langues_taikadai.html")

## Aller plus loin

Il manque des informations sur cette carte :
1. Associez à chaque marqueur un `PopUp` ([voir la documentation](https://python-visualization.github.io/folium/latest/reference.html#folium.map.Popup)) afin d'afficher la langue lorsque l'on clique avec la souris sur le marqueur.
2. L'identifiant, présent dans le dictionnaire pour chaque langue, peut permettre d'accéder à la page web associée à cette langue sur le portail Glottolog. En effet, le connaissant il suffit de construire l'URL de cette page de la manière suivante :<br> https://glottolog.org/resource/languoid/id/ennn1243<br>Ainsi, il est possible d'insérer à l'aide du `PopUp` un lien vers cette page.

> **Attention** : dans un notebook Python sous Visual Studio Code (VSC), les liens dans un Popup de folium ne peuvent pas être ouverts directement comme dans un navigateur traditionnel, car VSC ne gère pas le rendu des liens HTML de manière interactive (comme un véritable navigateur web). Cependant, le problème ne se pose pas en enregistrant la carte dans un fichier HTML et en l'ouvrant dans un navigateur.

---